In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
    "unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

alpaca_prompt = """Di bawah ini adalah instruksi yang menggambarkan sebuah tugas dengan input yang memberikan konteks lebih lanjut. Tuliskan respons yang tepat untuk menyelesaikan permintaan tersebut.

### Instruksi:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = """
Objektif: Tugas anda adalah menghasilkan urutan respons JSON untuk merencanakan tindakan lengan robot berdasarkan input pengguna. Jika tujuan tidak dapat dicapai dengan menggunakan instruksi yang disediakan dan objek yang tersedia, kembalikan pesan kesalahan.

Berikan objek JSON yang mengandung array "actions", diidentifikasi dengan key "actions".

Setiap aksi harus direpresentasikan sebagai objek dengan "command" dan "parameters" yang sesuai

Objek dan Koordinat yang Tersedia (x,y,z):
1. Balok ungu = (-86.59, 117.21, -122.30)
2. Balok kuning = (-168.94, -129.37, -68)
3. Balok biru = (152.76, 158.92, 6)

Perintah yang Tersedia:
1. move: Gerakkan lengan robot ke arah tertentu. Sertakan parameter "direction" dengan nilai "atas", "bawah", "depan", "belakang", "kiri", atau "kanan".
2. move_to: Gerakkan lengan robot ke koordinat tertentu. Sertakan parameter "x", "y", dan "z" untuk menentukan koordinat tujuan.
3. suction_cup: Aktifkan atau nonaktifkan penghisap. Gunakan parameter "action" dengan nilai "on" atau "off".
4. err_msg: Kembalikan pesan kesalahan jika tujuan pengguna tidak dapat tercapai dengan menggunakan objek dan perintah saat ini. Gunakan parameter "msg" dengan nilai "tidak dapat membuat rencana aksi dengan kondisi terkini".

Contoh Penggunaan Perintah:
"{"actions":[{"command":"move","parameters":{"direction":"atas"}},{"command":"move_to","parameters":{"x":-30.21,"y":233.32,"z":-40}},{"command":"suction_cup","parameters":{"action":"on"}},{"command":"err_msg","parameters":{"msg":"tidak dapat membuat rencana aksi dengan kondisi terkini"}}]}"

Instruksi Penggunaan:
1. Untuk memindahkan objek yang tersedia ke koordinat tertentu, aktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "on", kemudian gerakkan ke koordinat objek menggunakan perintah "move_to".
2. Berikan koordinat penempatan untuk tujuan pengguna menggunakan perintah "move_to".
3. Untuk melepaskan objek setelah menggunakan penyedot, nonaktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "off".
4. Untuk memindahkan robot secara lateral (misalnya ke kiri, kanan, depan, belakang, atas, depan), gunakan perintah "move" dengan arah yang sesuai.
5. Untuk memindahkan objek secara lateral (misalnya ke kiri, kanan, depan, belakang, atas, depan), pertama-tama gerakkan lengan robot ke koordinat objek menggunakan perintah "move_to", kemudian gunakan perintah "move" dengan arah yang sesuai.
6. Jika tujuan pengguna tidak dapat tercapai dengan perintah dan objek saat ini, gunakan perintah "err_msg".
    
"""

    inputs       = examples["input"]
    outputs      = examples["response"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# Get dataset
dataset = load_dataset("Aryaduta/test-data2", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

eval_data = load_dataset("Aryaduta/test-data2", split = "validation")
eval_data = eval_data.map(formatting_prompts_func, batched = True,)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    eval_dataset = eval_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_eval_batch_size=1,
        evaluation_strategy="steps",   
        per_device_train_batch_size = 1,
        eval_steps = 1000,
        warmup_steps = 2,
        num_train_epochs=3.0,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1000,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer.train()

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
"""
Objektif: Tugas anda adalah menghasilkan urutan respons JSON untuk merencanakan tindakan lengan robot berdasarkan input pengguna. Jika tujuan tidak dapat dicapai dengan menggunakan instruksi yang disediakan dan objek yang tersedia, kembalikan pesan kesalahan.

Berikan objek JSON yang mengandung array "actions", diidentifikasi dengan key "actions".

Setiap aksi harus direpresentasikan sebagai objek dengan "command" dan "parameters" yang sesuai

Objek dan Koordinat yang Tersedia (x,y,z):
1. Balok ungu = (-86.59, 117.21, -122.30)
2. Balok kuning = (-168.94, -129.37, -68)
3. Balok biru = (152.76, 158.92, 6)

Perintah yang Tersedia:
1. move: Gerakkan lengan robot ke arah tertentu. Sertakan parameter "direction" dengan nilai "atas", "bawah", "depan", "belakang", "kiri", atau "kanan".
2. move_to: Gerakkan lengan robot ke koordinat tertentu. Sertakan parameter "x", "y", dan "z" untuk menentukan koordinat tujuan.
3. suction_cup: Aktifkan atau nonaktifkan penghisap. Gunakan parameter "action" dengan nilai "on" atau "off".
5. err_msg: Kembalikan pesan kesalahan jika tujuan pengguna tidak dapat tercapai dengan menggunakan objek dan perintah saat ini. Gunakan parameter "msg" dengan nilai "tidak dapat membuat rencana aksi dengan kondisi terkini".

Contoh Penggunaan Perintah:
"{"actions":[{"command":"move","parameters":{"direction":"atas"}},{"command":"move_to","parameters":{"x":-30.21,"y":233.32,"z":-40}},{"command":"suction_cup","parameters":{"action":"on"}},{"command":"err_msg","parameters":{"msg":"tidak dapat membuat rencana aksi dengan kondisi terkini"}}]}"

Instruksi Penggunaan:
1. Untuk memindahkan objek yang tersedia ke koordinat tertentu, aktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "on", kemudian gerakkan ke koordinat objek menggunakan perintah "move_to".
2. Berikan koordinat penempatan untuk tujuan pengguna menggunakan perintah "move_to".
3. Untuk melepaskan objek setelah menggunakan penyedot, nonaktifkan penyedot terlebih dahulu menggunakan perintah "suction_cup" dengan "action" diatur ke "off".
4. Untuk memindahkan robot secara lateral (misalnya ke kiri, kanan, depan, belakang, atas, depan), gunakan perintah "move" dengan arah yang sesuai.
5. Untuk memindahkan objek secara lateral ((misalnya ke kiri, kanan, depan, belakang, atas, depan), pertama-tama gerakkan lengan robot ke koordinat objek menggunakan perintah "move_to", kemudian gunakan perintah "move" dengan arah yang sesuai.
6. Jika tujuan pengguna tidak dapat tercapai dengan perintah dan objek saat ini, gunakan perintah "err_msg".
      
"""
, # instruction
        "pindahkan balok biru di kanan balok kuning", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024)

In [ ]:
# Decode output and extract JSON response
decoded_output = tokenizer.batch_decode(outputs)[0]

# Find the start and end indices for the JSON response
start_marker = "### Response:"
end_marker = "<eos>"
start_index = decoded_output.find(start_marker) + len(start_marker)
end_index = decoded_output.find(end_marker, start_index)

# Extract the JSON response
json_response = decoded_output[start_index:end_index].strip()

# Print the JSON response
print(json_response)

In [ ]:
import os
hf_token = "XXXXXXXX" # Change to your HuggingFace Token

In [ ]:
!huggingface-cli login --token $hf_token

In [ ]:
model.push_to_hub_merged("model_name", tokenizer, save_method = "lora", token = hf_token) # Change your model name to your own liking